### IMPORT DEPENDENCIES

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.layers as layers

### Load Dataset

In [ ]:
train, info = tfds.load('tf_flowers', split='train[:90%]', with_info=True, as_supervised=True)
validation = tfds.load('tf_flowers', split='train[90%:]', as_supervised=True )

### Normalize and prepare dataset

In [ ]:
def normalize(image, label):
  img = tf.cast(image, tf.float32)
  img = img/255.0
  img = tf.image.resize(img, (224,224))
  return img, label

def applyPreprocessing(dataset, isNormalize=False):
  if(isNormalize):
    dataset = dataset.map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.shuffle(128)
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(64)
  return dataset


In [ ]:
train = applyPreprocessing(train, True)
validation = applyPreprocessing(validation, True)

### Create LearningRateScheduler Callback

In [ ]:
logdir = 'logs' # Path to the directory for logging
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

# Function to return the reduced the learning rate based on epoch
def lr_schedule(epoch):
  learning_rate = 0.2
  if epoch > 1:
    learning_rate = 0.02
  if epoch > 2:
    learning_rate = 0.01+epoch*0.001
  tf.summary.scalar('Learning Rate', data=learning_rate, step=epoch)
  return learning_rate

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

### Define custom model

In [ ]:
model = tf.keras.Sequential([
                             #layers.Reshape((256,256,3)),
                             tf.keras.layers.InputLayer(input_shape=(224,224,3)),
                              layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
                             layers.MaxPool2D(),
                             layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
                             layers.MaxPool2D(),
                             layers.Dropout(0.25),
                             layers.Conv2D(filters=128, kernel_size=3, activation='relu'),
                             layers.MaxPool2D(),
                             layers.Dropout(0.25),
                             layers.Flatten(),
                             layers.Dense(128, activation='relu'),
                             layers.Dropout(0.25),
                             layers.Dense(5, activation='softmax')
])
model.summary()

In [ ]:
### Train the model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train, validation_data=validation, epochs=3, callbacks=[lr_callback, tensorboard_callback])

In [ ]:
### Use the tensorflow hub to load the pretrained mobilenet and do transfer learning

In [ ]:
import tensorflow_hub as hub
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url)

logdir = 'logs/transfer_training'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

feature_extractor_layer.trainable = False
model_2 = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(224,224, 3)),
  feature_extractor_layer,
  layers.Dense(5, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model_2.build((None,255,255, 3))

model_2.compile(optimizer='adam', loss='binary_crossentropy')

model_2.fit(train, validation_data=validation, epochs=3, callbacks=[lr_callback, tensorboard_callback])

### Visualize the tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/